# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
`
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)

# Não há duplicados
sinasc.drop_duplicates().shape
sinasc.isna().sum()
is_duplicated = sinasc.duplicated()
is_duplicated.sum()

(27028, 69)


0

In [2]:
# 2) seu código aqui
missing = sinasc.isna().sum()
missing

,0
ORIGEM,0
CODESTAB,115
CODMUNNASC,0
LOCNASC,0
IDADEMAE,0
...,...
munResUf,0
munResLat,1
munResLon,1
munResAlt,1


In [3]:
# 3) seu código aqui
sinasc_filtrado = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_filtrado

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [4]:
# 4) seu código aqui
print(sinasc_filtrado.isna().sum())

# Removendo registros com Apgar5 não preenchido
sinasc_filtrado.dropna(subset=['APGAR5'], inplace=True)

# Contando os valores ausentes após a remoção
missing = sinasc_filtrado.isna().sum()

# Exibindo o número de linhas após a remoção e os valores ausentes por coluna
print(f"Número de linhas após a remoção: {sinasc_filtrado.shape[0]}")
print("Valores ausentes por coluna após a remoção:")
print(missing)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64
Número de linhas após a remoção: 26925
Valores ausentes por coluna após a remoção:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


<ipython-input-4-46a5f01095a4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_filtrado.dropna(subset=['APGAR5'], inplace=True)


In [5]:
# 5) seu código aqui
sinasc_filtrado['ESTCIVMAE'].fillna(9, inplace=True)
sinasc_filtrado['CONSULTAS'].fillna(9, inplace=True)

# Contando os valores ausentes antes da remoção
print(sinasc_filtrado.isna().sum())

# Removendo registros com Apgar5 não preenchido
sinasc_filtrado.dropna(subset=['APGAR5'], inplace=True)

# Contando os valores ausentes após a remoção
missing = sinasc_filtrado.isna().sum()

# Exibindo o número de linhas após a remoção e os valores ausentes por coluna
print(f"Número de linhas após a remoção: {sinasc_filtrado.shape[0]}")
print("Valores ausentes por coluna após a remoção:")
print(missing)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64
Número de linhas após a remoção: 26925
Valores ausentes por coluna após a remoção:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


<ipython-input-5-30e4d0026c52>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_filtrado['ESTCIVMAE'].fillna(9, inplace=True)
<ipython-input-5-30e4d0026c52>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_filtrado['CONSULTAS'].fillna(9, inplace=True)
<ipython-input-5-30e4d0026c52>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_filtrado.dropna(subset=['APGAR5'], inplace=True)


In [6]:
# 6) Seu código aqui
sinasc_filtrado['QTDFILVIVO'].fillna(0, inplace=True)

# Removendo registros com Apgar5 não preenchido
sinasc_filtrado.dropna(subset=['APGAR5'], inplace=True)

# Contando os valores ausentes após a remoção
missing = sinasc_filtrado.isna().sum()

# Exibindo o número de linhas após a remoção e os valores ausentes por coluna
print(f"Número de linhas após a remoção: {sinasc_filtrado.shape[0]}")
print("Valores ausentes por coluna após a remoção:")
print(missing)

Número de linhas após a remoção: 26925
Valores ausentes por coluna após a remoção:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


<ipython-input-6-ea2aa717bd9f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_filtrado['QTDFILVIVO'].fillna(0, inplace=True)
<ipython-input-6-ea2aa717bd9f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_filtrado.dropna(subset=['APGAR5'], inplace=True)


In [7]:
# 7) seu código aqui
# Substituir os valores faltantes restantes por -1
missing_values_before = sinasc.isna().sum().sum()

sinasc.fillna(-1, inplace=True)

# Contar quantas linhas foram substituídas
missing_values_after = sinasc.isna().sum().sum()
rows_filled = missing_values_before - missing_values_after

rows_filled

121594

In [14]:
# 8) seu código aqui
apgar5_categorias = {
    (8, 10): 'normal',
    (6, 7): 'asfixia leve',
    (4, 5): 'asfixia moderada',
    (0, 3): 'asfixia severa'
}

sinasc_filtrado['APGAR5_CATEGORIA'] = pd.cut(
    sinasc_filtrado['APGAR5'],
    bins=[0, 3, 5, 7, 10],
    labels=['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal'],
    include_lowest=True
)

# Exibindo as primeiras linhas do DataFrame com a nova coluna
sinasc_filtrado.head()

<ipython-input-14-6c253ffa8e52>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_filtrado['APGAR5_CATEGORIA'] = pd.cut(


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CATEGORIA
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


In [17]:
# 9) seu código aqui
inplaceter = True
sinasc_filtrado.columns = sinasc_filtrado.columns.str.lower().str.replace(' ', '_')
sinasc_filtrado.head()


,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,apgar5_categoria
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
